In [2]:
import sys
sys.path.append("../skillExtract/")
import pandas as pd
%load_ext autoreload
%autoreload 2


test = pd.read_csv("https://raw.githubusercontent.com/jensjorisdecorte/Skill-Extraction-benchmark/main/tech_test_annotations.csv")

In [2]:
test.sample(5)

,sentence,span,sub_span,label
394,As a part of the Games Team you will have the ...,work closely with other programmers designers ...,work closely with other designers,collaborate with designers
279,Knowledge of tools procedures and programming ...,tools procedures and programming languages for...,programming languages for test automation,tools for ICT test automation
619,This along with the capture & analysis of requ...,capture & analysis of requirements,capture requirements,LABEL NOT PRESENT
236,Coding/scripting and basic DevOps skills to bu...,DevOps,NaN,DevOps
749,Experience with development frameworks ( SCRUM...,SCRUM/Agile ),NaN,LABEL NOT PRESENT


### Laoding records

In [3]:
multilabel = test.groupby("sentence").agg({"label": lambda x : x.tolist() , "span": lambda x : x.tolist()}).reset_index()
annotated_records = multilabel.to_dict(orient='records')

### Loading word emebdding model

In [4]:
from transformers import (AutoModel, AutoTokenizer)

word_emb = "jjzha/jobbert-base-cased"
word_emb_model = AutoModel.from_pretrained(word_emb)
word_emb_tokenizer = AutoTokenizer.from_pretrained(word_emb)

Some weights of BertModel were not initialized from the model checkpoint at jjzha/jobbert-base-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Load the taxonomy

In [5]:
from utils import load_taxonomy
from api_key import API_KEY
class Args():
    def __init__(self):
        self.taxonomy = "../../../esco/skills_en.csv" 

        ## RELATED TO PROMPT CREATION
        self.datapath = "remote-annotated-en"
        self.candidates_method="rules"
        self.shots = 6
        self.prompt_type = "inde"
        
        ## RELATED TO CHAT GPT GENERATION
        self.max_tokens = 200      ## ?? default value but JobBERT suposedly takes 512
        self.api_key = API_KEY
        self.model = "gpt-3.5-turbo"
        self.temperature = 0       ## default val
        self.top_p = 1             ## default val
        self.frequency_penalty = 0 ## default val
        self.presence_penalty = 0  ## default val

args = Args()

In [6]:
cols = ["conceptType", "skillType", "reuseLevel", "preferredLabel", "altLabels", "description"]
tax = pd.read_csv(args.taxonomy)[cols]
tax = tax.reset_index()
tax.columns = ["unique_id",
               "Type Level 1",
               "Type Level 2",
               "Type Level 3",
               "name",
               "Example",
               "Definition"
               ]
tax["name+definition"] = tax[["name", "Definition"]].apply(lambda x : x["name"] + " : " + x["Definition"], axis=1)

### Generate embedded taxonomy

In [7]:
import pickle
from utils import embed_taxonomy

emb_sh = "_jbEn"
try:
    print(f"Loading embedded taxonomy for {word_emb}")
    with open(f"../../data/taxonomy/taxonomy_embeddings{emb_sh}.pkl", "rb") as f:
        emb_tax = pickle.load(f)
except:
    print(f"Loading failed, generating embedded taxonomy for {word_emb}")
    emb_tax = embed_taxonomy(tax, word_emb_model, word_emb_tokenizer)
    with open(f"../../data/taxonomy/taxonomy_embeddings{emb_sh}.pkl", "wb") as f:
        pickle.dump(emb_tax, f)

Loading embedded taxonomy for jjzha/jobbert-base-cased


### Running through the annotated records

In [34]:
from tqdm.notebook import tqdm
from utils import (OPENAI,
                   Splitter,
                   select_candidates_from_taxonomy)
from api_key import API_KEY

extraction_cost = 0
matching_cost = 0
ress = []
for i, annotated_record in tqdm(enumerate(annotated_records)):
    if(i>20):
        break
    print(annotated_record)

    ## EXTRACTION
    api = OPENAI(args, [annotated_record])
    sentences_res_list, cost = api.do_prediction("extraction")
    extraction_cost += cost
    

    ## CANDIDATE SELECTION
    if "extracted_skills" in sentences_res_list[0]:
        splitter = Splitter()
        max_candidates = 10
        for idxx, sample in enumerate(sentences_res_list):
            sample = select_candidates_from_taxonomy(
                sample,
                tax,
                splitter,
                word_emb_model,
                word_emb_tokenizer,
                max_candidates,
                method=args.candidates_method,
                emb_tax=None if args.candidates_method == "rules" else emb_tax,
            )
            sentences_res_list[idxx] = sample


    if("skill_candidates" in sentences_res_list[0]):
        api = OPENAI(args, sentences_res_list)
        sentences_res_list, cost = api.do_prediction("matching")
        matching_cost += cost



    ress.append(sentences_res_list)

0it [00:00, ?it/s]

{'sentence': '( Cypress for frontend is a must have pytest for backend and truffle for Solidity is a good addition) .', 'label': ['LABEL NOT PRESENT', 'LABEL NOT PRESENT', 'LABEL NOT PRESENT', 'Solidity'], 'span': ['Cypress', 'pytest', 'truffle', 'Solidity'], 'extracted_skills': ['pytest', 'truffle'], 'skill_candidates': {'pytest': [{'unique_id': 1478, 'Type Level 2': 'knowledge', 'name+definition': 'Estonian : The\xa0Estonian language. Estonian is an official and working language of the EU.'}, {'unique_id': 1765, 'Type Level 2': 'skill/competence', 'name+definition': 'keep up with the latest information systems solutions : Gather the latest information on existing information systems solutions which integrate software and hardware, as well as network components.'}, {'unique_id': 1936, 'Type Level 2': 'skill/competence', 'name+definition': 'plan tanning finishing operations : Plan the finishing operations to produce leather. Adjust the formulation of finishing operation according to ea

100%|██████████| 1/1 [00:01<00:00,  1.03s/it]


Costs:  extraction 0.000642
extracted skill: pytest
checking for matches in name+definition
checking for matches in example
checking for matches in subwords
extracted skill: truffle
checking for matches in name+definition


100%|██████████| 1/1 [00:00<00:00,  1.29it/s]


...

Costs:  matching 0.0021950000000000003
{'sentence': '* Knowledge of best practices in DevOps and CI/CD automation', 'label': ['DevOps', 'LABEL NOT PRESENT'], 'span': ['DevOps', 'CI/CD automation']}


100%|██████████| 1/1 [00:00<00:00,  1.63it/s]


Costs:  extraction 0.0006105
extracted skill: devops
checking for matches in name+definition
extracted skill: ci/cd automation
checking for matches in name+definition
checking for matches in example
checking for matches in subwords


100%|██████████| 1/1 [00:00<00:00,  1.50it/s]

Costs:  matching 0.0010985


In [51]:
for i, [res] in enumerate(ress):
    print(res["sentence"])
    print(res["label"])
    print(res["span"])
    for extracted_skill in res["matched_skills"].keys():
        print("- ", extracted_skill, " > ", res["matched_skills"][extracted_skill]["name+definition"])
    print("-"*100)


( Cypress for frontend is a must have pytest for backend and truffle for Solidity is a good addition) .
['LABEL NOT PRESENT', 'LABEL NOT PRESENT', 'LABEL NOT PRESENT', 'Solidity']
['Cypress', 'pytest', 'truffle', 'Solidity']
-  pytest  >  demonstrate proficiency in packaging standards : Work in accordance with the latest developments in domestic and international packaging standards and procedures.
----------------------------------------------------------------------------------------------------
* Ability to analyse requirements and interact directly with business users
['analyse business requirements', 'communicate with stakeholders']
['analyse requirements', 'interact directly with business users']
-  analyse requirements  >  analyse requirements for moving goods : Analyse goods to be relocated and their moving requirements. Check requirements and prepare actions to ensure appropriate transport of goods.
-  business users  >  propose ICT solutions to business problems : Suggest how

### Metrics